**Predicting Next Purchase Day**

In [0]:
#Import libraries
from __future__ import division
from datetime import datetime, timedelta,date
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler, StandardScalerModel
from pyspark.ml.evaluation import ClusteringEvaluator
#do not show warnings
import warnings
warnings.filterwarnings("ignore")
#import plotly for visualization
# import chart_studio.plotly  as py
# import plotly.graph_objs as go
# from plotly.offline import plot
import pylab as pl
# import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import pyspark.sql.functions as funcs
from pyspark.sql.functions import *
from pyspark.sql.functions import lag
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler, StandardScalerModel
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import to_json, struct
from pyspark.sql import Window
from pyspark.sql import SparkSession   # pyspark session
spark = SparkSession.builder.appName("Hive test").enableHiveSupport().getOrCreate()  #create a spark object and enable hive
sqlContext.setConf("spark.sql.parquet.writeLegacyFormat", "true") # To enable all parquet files with same datatypes 

# Using Carrefour_v1 database 
spark.sql('use cpr_officedepot')

Out[5]: DataFrame[]

In [0]:
# Function to plot Elbow graph and silhouette score to find the optimal clusters
def finding_optimal_clusters(data):
    try:
        cost = list()
        for k in range(2,10):
            kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
            kmeans_model = kmeans.fit(data)
#             cost.append(kmeans_model.computeCost(data)/k)
            
            # Make predictions to see silhouette score
            pred = kmeans_model.transform(data)
            # Evaluate clustering by computing Silhouette score
            evaluator = ClusteringEvaluator()
            silhouette = evaluator.evaluate(pred)
            print("For " +str(k)+ " clusters, Silhouette with squared euclidean distance = " + str(silhouette))
        # Plot the cost
#         df_cost = pd.DataFrame(cost)
#         df_cost.columns = ["cost"]
#         new_col = [2,3,4,5,6,7,8,9]
#         df_cost.insert(0, 'cluster', new_col)
#         plt.figure(figsize=(20,8))
#         pl.plot(df_cost.cluster, df_cost.cost)
#         pl.xlabel('Number of Clusters')
#         pl.ylabel('Withing set sum of square')
#         pl.title('Elbow Curve')
#         display(pl.show())
    except Exception as e:
           print(" Exception in finding_optimal_clusters function")
           print(e)    
# function to scale the variables
def vector_scaling(raw_data,*var):
    try:
        assembler = VectorAssembler( inputCols= [i for i in var], outputCol="features")
        scaling_data = assembler.transform(raw_data)
        scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)
        scalerModel = scaler.fit(scaling_data)
        scaling_data = scalerModel.transform(scaling_data).drop('features').withColumnRenamed('scaledFeatures', 'features')
        return scaling_data
    except Exception as e:
           print("Exception in scaling function")
           print(e)
# Function for K-Means clustering
def clustering(data,k):
    try:
        kmeans_final = KMeans().setK(k).setSeed(1) 
        final_model = kmeans_final.fit(data.select("features"))
        transformed = final_model.transform(data)
        rows = transformed.collect()
        output = sqlContext.createDataFrame(rows)
        return output
    except Exception as e:
           print(" Exception in Clustering function")
           print(e)   
# function to scale the variables
def scaling(raw_data,var,cols):
    try:
        assembler = VectorAssembler( inputCols= [i for i in var if i not in cols], outputCol="features")
        scaling_data = assembler.transform(raw_data)
        scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)
        scalerModel = scaler.fit(scaling_data)
        scaling_data = scalerModel.transform(scaling_data).drop('features').withColumnRenamed('scaledFeatures', 'features')
        return scaling_data
    except Exception as e:
           print("Exception in scaling function")
           print(e)
#function to implement linear regression          
def linear_regression(data,label):
    lr = LinearRegression(featuresCol = 'features', labelCol=label, maxIter=10, regParam=0.3, elasticNetParam=0.8)
    lr_model = lr.fit(data)
    lr_predictions = lr_model.transform(data)
    return lr_predictions.select("customer_id",col("prediction").alias(label))                    

In [0]:
# Office Depot - Reading 4 years of data and splitting 3.2 years as behaviour_data and last 8 months as future data to implement predicitons

## next 3 purchase prediction 
behavior_data = spark.sql("""
select transaction_date, 
customer_id, 
sum(gross_sales) as revenue from gm_product_profit_fact
WHERE 
transaction_date <= '2018-08-15'
GROUP BY 
transaction_date,
customer_id""")

future_data = spark.sql("""
select transaction_date, customer_id, sum(gross_sales) as revenue from gm_product_profit_fact
where transaction_date > '2018-08-15'
group by transaction_date,
customer_id""")

#finding unique customers to create user level feature set
tx_user = behavior_data.select("customer_id").distinct()

#Sorting the data
tx_next_purchases = future_data.select('customer_id','transaction_date').orderBy('customer_id','transaction_date')

#shifting last 5 purchase dates
w = Window().partitionBy('customer_id').orderBy('transaction_date')
tx_next_purchases=tx_next_purchases.withColumn('prev_transaction_date1', lag('transaction_date').over(w))
tx_next_purchases=tx_next_purchases.withColumn('prev_transaction_date2', lag('transaction_date',count=2).over(w))
tx_next_purchases=tx_next_purchases.withColumn('prev_transaction_date3', lag('transaction_date',count=3).over(w))


#create a dataframe with customer id and last purchase date from behavior_data
tx_last_purchase = behavior_data.groupby('customer_id').agg({'transaction_date' : 'max'}).toDF('customer_id','max_purchase_date')

#merging last purchase and next purchase dataframes
tx_purchase_dates = tx_next_purchases.join(tx_last_purchase,on='customer_id',how='left')

# Dropping null values
tx_purchase_dates = tx_purchase_dates.dropna()
# Finding the minimum value from purchase dates dataframe
tx_first_purchase = tx_purchase_dates.groupBy('customer_id').agg({'transaction_date' : 'min'}).toDF('customer_id','transaction_date')
#Joining two dataframes
tx_first_purchase = tx_purchase_dates.join(tx_first_purchase,on=['customer_id','transaction_date'],how='inner')


#Sorting the data
tx_first_purchase = tx_first_purchase.orderBy('customer_id','transaction_date')

# Deriving 5 next purchase days
tx_first_purchase=tx_first_purchase.withColumn('next_purchase_day_1st',  funcs.datediff(tx_first_purchase['prev_transaction_date3'] ,tx_first_purchase['max_purchase_date']))
tx_first_purchase=tx_first_purchase.withColumn('next_purchase_day_2nd',  funcs.datediff(tx_first_purchase['prev_transaction_date2'] ,tx_first_purchase['max_purchase_date']))
tx_first_purchase=tx_first_purchase.withColumn('next_purchase_day_3rd',  funcs.datediff(tx_first_purchase['prev_transaction_date1'] ,tx_first_purchase['max_purchase_date']))

#merge with tx_user 
tx_user = tx_user.join(tx_first_purchase[['customer_id','next_purchase_day_1st','next_purchase_day_2nd','next_purchase_day_3rd']],on='customer_id',how='inner') 

#get max purchase date for Recency and create a dataframe
tx_max_purchase = behavior_data.groupby('customer_id').agg({'transaction_date' : 'max'}).toDF('customer_id','max_purchase_date')

#find the recency in days and add it to tx_user
from pyspark.sql.functions import max
tx_max_purchase=tx_max_purchase.withColumn('recency',  funcs.datediff(max('max_purchase_date').over(Window.partitionBy()) ,'max_purchase_date'))
tx_user = tx_user.join(tx_max_purchase[['customer_id','recency']],on='customer_id',how='inner') 

#get total purchases for frequency scores
tx_frequency = behavior_data.groupby('customer_id').agg({'transaction_date' : 'count'}).toDF('customer_id','frequency')

#add frequency column to tx_user
tx_user = tx_user.join(tx_frequency[['customer_id','frequency']],on='customer_id',how='inner')
#calculate revenue value, create a dataframe with it
tx_revenue=behavior_data.groupby('customer_id').agg({'revenue' : 'sum'}).toDF('customer_id','revenue')
#add Revenue column to tx_user
tx_user = tx_user.join(tx_revenue[['customer_id','revenue']],on='customer_id',how='inner')

tx_day_order = behavior_data.select('customer_id','transaction_date').orderBy('customer_id','transaction_date')
#Customer can do multiple purchases in a day and difference will become 0 for those. So we are removing the duplicates
tx_day_order = tx_day_order.drop_duplicates(subset=['customer_id','transaction_date'])

from pyspark.sql.functions import lag
w = Window().partitionBy('customer_id').orderBy('transaction_date')
tx_day_order = tx_day_order.withColumn('Prev_transaction_date',lag('transaction_date').over(w))
tx_day_order = tx_day_order.withColumn('t2_transaction_date',lag('transaction_date',count=2).over(w))
tx_day_order = tx_day_order.withColumn('t3_transaction_date',lag('transaction_date',count=3).over(w))

# creating new columns with the dates of last 5 purchases from behavioral data
tx_day_order = tx_day_order.withColumn('day_diff',funcs.datediff(tx_day_order['transaction_date'] ,tx_day_order['prev_transaction_date']))
tx_day_order = tx_day_order.withColumn('day_diff2',funcs.datediff(tx_day_order['transaction_date'] ,tx_day_order['t2_transaction_date']))
tx_day_order = tx_day_order.withColumn('day_diff3',funcs.datediff(tx_day_order['transaction_date'] ,tx_day_order['t3_transaction_date']))


#finding mean and standard deviation of the difference between purchases in days
tx_day_diff = tx_day_order.groupby('customer_id').agg(funcs.mean('day_diff'), funcs.stddev('day_diff')).toDF('customer_id','day_diff_mean','day_diff_std')

# We are trying to keep customers who has >5 purchases
tx_day_order_last = tx_day_order.groupBy('customer_id').agg({'transaction_date' : 'max'}).toDF('customer_id','transaction_date')
tx_day_order_last = tx_day_order_last.join(tx_day_order, (tx_day_order.customer_id==tx_day_order_last.customer_id) & (tx_day_order.transaction_date==tx_day_order_last.transaction_date), 'inner').select(tx_day_order.customer_id,'day_diff','day_diff2','day_diff3')
tx_day_order_last = tx_day_order_last.join(tx_day_diff, on = 'customer_id', how='inner')

# Removing null values
tx_day_order_last = tx_day_order_last.dropna()

# Joining to tx_user dataframe
tx_user = tx_user.join(tx_day_order_last, on = 'customer_id', how='inner')


#display(tx_user)

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-338931858627303> in <module> 
 27 w = Window ( ) . partitionBy ( 'customer_id' ) . orderBy ( 'transaction_date' ) 
 28 tx_next_purchases = tx_next_purchases . withColumn ( 'prev_transaction_date1' , lag ( 'transaction_date' ) . over ( w ) ) 
 ---> 29 tx_next_purchases = tx_next_purchases . withColumn ( 'prev_transaction_date2' , lag ( 'transaction_date' , count = 2 ) . over ( w ) ) 
 30 tx_next_purchases = tx_next_purchases . withColumn ( 'prev_transaction_date3' , lag ( 'transaction_date' , count = 3 ) . over ( w ) ) 
 31 

 TypeError : lag() got an unexpected keyword argument 'count'

In [0]:
# Carrefour - Reading 2 years of data and splitting 1.6 years as behaviour_data and last 6 months as future data to implement predicitons

# change dates in next two dataframes according to data splitting 
'''
behavior_data = spark.sql("""
select transaction_date, customer_id, sum(gross_sales) as revenue from gm_product_profit_fact
where transaction_date <= '2020-02-29'
group by transaction_date,customer_id""")

future_data = spark.sql("""
select transaction_date, customer_id, sum(gross_sales) as revenue from gm_product_profit_fact
where transaction_date > '2020-02-29'
group by transaction_date,customer_id""")

#finding unique customers to create user level feature set
tx_user = behavior_data.select("customer_id").distinct()

#Sorting the data
tx_next_purchases = future_data.select('customer_id','transaction_date').orderBy('customer_id','transaction_date')

#shifting last 5 purchase dates
w = Window().partitionBy('customer_id').orderBy('transaction_date')
tx_next_purchases=tx_next_purchases.withColumn('prev_transaction_date1', lag('transaction_date').over(w))
tx_next_purchases=tx_next_purchases.withColumn('prev_transaction_date2', lag('transaction_date',count=2).over(w))
tx_next_purchases=tx_next_purchases.withColumn('prev_transaction_date3', lag('transaction_date',count=3).over(w))
tx_next_purchases=tx_next_purchases.withColumn('prev_transaction_date4', lag('transaction_date',count=4).over(w))
tx_next_purchases=tx_next_purchases.withColumn('prev_transaction_date5', lag('transaction_date',count=5).over(w))

#create a dataframe with customer id and last purchase date from behavior_data
tx_last_purchase = behavior_data.groupby('customer_id').agg({'transaction_date' : 'max'}).toDF('customer_id','max_purchase_date')

#merging last purchase and next purchase dataframes
tx_purchase_dates = tx_next_purchases.join(tx_last_purchase,on='customer_id',how='left')

# Dropping null values
tx_purchase_dates = tx_purchase_dates.dropna()
# Finding the minimum value from purchase dates dataframe
tx_first_purchase = tx_purchase_dates.groupBy('customer_id').agg({'transaction_date' : 'min'}).toDF('customer_id','transaction_date')
#Joining two dataframes
tx_first_purchase = tx_purchase_dates.join(tx_first_purchase,on=['customer_id','transaction_date'],how='inner')


#Sorting the data
tx_first_purchase = tx_first_purchase.orderBy('customer_id','transaction_date')

# Deriving 5 next purchase days
tx_first_purchase=tx_first_purchase.withColumn('1st_next_purchase_day',  funcs.datediff(tx_first_purchase['prev_transaction_date5'] ,tx_first_purchase['max_purchase_date']))
tx_first_purchase=tx_first_purchase.withColumn('2nd_next_purchase_day',  funcs.datediff(tx_first_purchase['prev_transaction_date4'] ,tx_first_purchase['max_purchase_date']))
tx_first_purchase=tx_first_purchase.withColumn('3rd_next_purchase_day',  funcs.datediff(tx_first_purchase['prev_transaction_date3'] ,tx_first_purchase['max_purchase_date']))
tx_first_purchase=tx_first_purchase.withColumn('4th_next_purchase_day',  funcs.datediff(tx_first_purchase['prev_transaction_date2'] ,tx_first_purchase['max_purchase_date']))
tx_first_purchase=tx_first_purchase.withColumn('5th_next_purchase_day',  funcs.datediff(tx_first_purchase['prev_transaction_date1'] ,tx_first_purchase['max_purchase_date']))

#merge with tx_user 
tx_user = tx_user.join(tx_first_purchase[['customer_id','1st_next_purchase_day','2nd_next_purchase_day','3rd_next_purchase_day','4th_next_purchase_day','5th_next_purchase_day']],on='customer_id',how='inner') 

#get max purchase date for Recency and create a dataframe
tx_max_purchase = behavior_data.groupby('customer_id').agg({'transaction_date' : 'max'}).toDF('customer_id','max_purchase_date')

#find the recency in days and add it to tx_user
from pyspark.sql.functions import max
tx_max_purchase=tx_max_purchase.withColumn('recency',  funcs.datediff(max('max_purchase_date').over(Window.partitionBy()) ,'max_purchase_date'))
tx_user = tx_user.join(tx_max_purchase[['customer_id','recency']],on='customer_id',how='inner') 

#get total purchases for frequency scores
tx_frequency = behavior_data.groupby('customer_id').agg({'transaction_date' : 'count'}).toDF('customer_id','frequency')

#add frequency column to tx_user
tx_user = tx_user.join(tx_frequency[['customer_id','frequency']],on='customer_id',how='inner')
#calculate revenue value, create a dataframe with it
tx_revenue=behavior_data.groupby('customer_id').agg({'revenue' : 'sum'}).toDF('customer_id','revenue')
#add Revenue column to tx_user
tx_user = tx_user.join(tx_revenue[['customer_id','revenue']],on='customer_id',how='inner')

tx_day_order = behavior_data.select('customer_id','transaction_date').orderBy('customer_id','transaction_date')
#Customer can do multiple purchases in a day and difference will become 0 for those. So we are removing the duplicates
tx_day_order = tx_day_order.drop_duplicates(subset=['customer_id','transaction_date'])

from pyspark.sql.functions import lag
w = Window().partitionBy('customer_id').orderBy('transaction_date')
tx_day_order = tx_day_order.withColumn('Prev_transaction_date',lag('transaction_date').over(w))
tx_day_order = tx_day_order.withColumn('t2_transaction_date',lag('transaction_date',count=2).over(w))
tx_day_order = tx_day_order.withColumn('t3_transaction_date',lag('transaction_date',count=3).over(w))
tx_day_order = tx_day_order.withColumn('t4_transaction_date',lag('transaction_date',count=4).over(w))
tx_day_order = tx_day_order.withColumn('t5_transaction_date',lag('transaction_date',count=5).over(w))

# creating new columns with the dates of last 5 purchases from behavioral data
tx_day_order = tx_day_order.withColumn('day_diff',funcs.datediff(tx_day_order['transaction_date'] ,tx_day_order['prev_transaction_date']))
tx_day_order = tx_day_order.withColumn('day_diff2',funcs.datediff(tx_day_order['transaction_date'] ,tx_day_order['t2_transaction_date']))
tx_day_order = tx_day_order.withColumn('day_diff3',funcs.datediff(tx_day_order['transaction_date'] ,tx_day_order['t3_transaction_date']))
tx_day_order = tx_day_order.withColumn('day_diff4',funcs.datediff(tx_day_order['transaction_date'] ,tx_day_order['t4_transaction_date']))
tx_day_order = tx_day_order.withColumn('day_diff5',funcs.datediff(tx_day_order['transaction_date'] ,tx_day_order['t5_transaction_date']))

#finding mean and standard deviation of the difference between purchases in days
tx_day_diff = tx_day_order.groupby('customer_id').agg(funcs.mean('day_diff'), funcs.stddev('day_diff')).toDF('customer_id','day_diff_mean','day_diff_std')

# We are trying to keep customers who has >5 purchases
tx_day_order_last = tx_day_order.groupBy('customer_id').agg({'transaction_date' : 'max'}).toDF('customer_id','transaction_date')
tx_day_order_last = tx_day_order_last.join(tx_day_order, (tx_day_order.customer_id==tx_day_order_last.customer_id) & (tx_day_order.transaction_date==tx_day_order_last.transaction_date), 'inner').select(tx_day_order.customer_id,'day_diff','day_diff2','day_diff3','day_diff4','day_diff5')
tx_day_order_last = tx_day_order_last.join(tx_day_diff, on = 'customer_id', how='inner')

# Removing null values
tx_day_order_last = tx_day_order_last.dropna()

# Joining to tx_user dataframe
tx_user = tx_user.join(tx_day_order_last, on = 'customer_id', how='inner')
'''

Out[19]: '\nbehavior_data = spark.sql("""\nselect transaction_date, customer_id, sum(gross_sales) as revenue from gm_product_profit_fact\nwhere transaction_date <= \'2020-02-29\'\ngroup by transaction_date,customer_id""")\n\nfuture_data = spark.sql("""\nselect transaction_date, customer_id, sum(gross_sales) as revenue from gm_product_profit_fact\nwhere transaction_date > \'2020-02-29\'\ngroup by transaction_date,customer_id""")\n\n#finding unique customers to create user level feature set\ntx_user = behavior_data.select("customer_id").distinct()\n\n#Sorting the data\ntx_next_purchases = future_data.select(\'customer_id\',\'transaction_date\').orderBy(\'customer_id\',\'transaction_date\')\n\n#shifting last 5 purchase dates\nw = Window().partitionBy(\'customer_id\').orderBy(\'transaction_date\')\ntx_next_purchases=tx_next_purchases.withColumn(\'prev_transaction_date1\', lag(\'transaction_date\').over(w))\ntx_next_purchases=tx_next_purchases.withColumn(\'prev_transaction_date2\', lag(\'transaction_date\',count=2).over(w))\ntx_next_purchases=tx_next_purchases.withColumn(\'prev_transaction_date3\', lag(\'transaction_date\',count=3).over(w))\ntx_next_purchases=tx_next_purchases.withColumn(\'prev_transaction_date4\', lag(\'transaction_date\',count=4).over(w))\ntx_next_purchases=tx_next_purchases.withColumn(\'prev_transaction_date5\', lag(\'transaction_date\',count=5).over(w))\n\n#create a dataframe with customer id and last purchase date from behavior_data\ntx_last_purchase = behavior_data.groupby(\'customer_id\').agg({\'transaction_date\' : \'max\'}).toDF(\'customer_id\',\'max_purchase_date\')\n\n#merging last purchase and next purchase dataframes\ntx_purchase_dates = tx_next_purchases.join(tx_last_purchase,on=\'customer_id\',how=\'left\')\n\n# Dropping null values\ntx_purchase_dates = tx_purchase_dates.dropna()\n# Finding the minimum value from purchase dates dataframe\ntx_first_purchase = tx_purchase_dates.groupBy(\'customer_id\').agg({\'transaction_date\' : \'min\'}).toDF(\'customer_id\',\'transaction_date\')\n#Joining two dataframes\ntx_first_purchase = tx_purchase_dates.join(tx_first_purchase,on=[\'customer_id\',\'transaction_date\'],how=\'inner\')\n\n\n#Sorting the data\ntx_first_purchase = tx_first_purchase.orderBy(\'customer_id\',\'transaction_date\')\n\n# Deriving 5 next purchase days\ntx_first_purchase=tx_first_purchase.withColumn(\'1st_next_purchase_day\', funcs.datediff(tx_first_purchase[\'prev_transaction_date5\'] ,tx_first_purchase[\'max_purchase_date\']))\ntx_first_purchase=tx_first_purchase.withColumn(\'2nd_next_purchase_day\', funcs.datediff(tx_first_purchase[\'prev_transaction_date4\'] ,tx_first_purchase[\'max_purchase_date\']))\ntx_first_purchase=tx_first_purchase.withColumn(\'3rd_next_purchase_day\', funcs.datediff(tx_first_purchase[\'prev_transaction_date3\'] ,tx_first_purchase[\'max_purchase_date\']))\ntx_first_purchase=tx_first_purchase.withColumn(\'4th_next_purchase_day\', funcs.datediff(tx_first_purchase[\'prev_transaction_date2\'] ,tx_first_purchase[\'max_purchase_date\']))\ntx_first_purchase=tx_first_purchase.withColumn(\'5th_next_purchase_day\', funcs.datediff(tx_first_purchase[\'prev_transaction_date1\'] ,tx_first_purchase[\'max_purchase_date\']))\n\n#merge with tx_user \ntx_user = tx_user.join(tx_first_purchase[[\'customer_id\',\'1st_next_purchase_day\',\'2nd_next_purchase_day\',\'3rd_next_purchase_day\',\'4th_next_purchase_day\',\'5th_next_purchase_day\']],on=\'customer_id\',how=\'inner\') \n\n#get max purchase date for Recency and create a dataframe\ntx_max_purchase = behavior_data.groupby(\'customer_id\').agg({\'transaction_date\' : \'max\'}).toDF(\'customer_id\',\'max_purchase_date\')\n\n#find the recency in days and add it to tx_user\nfrom pyspark.sql.functions import max\ntx_max_purchase=tx_max_purchase.withColumn(\'recency\', funcs.datediff(max(\'max_purchase_date\').over(Window.partitionBy()) ,\'max_purchase_date\'))\ntx_user = tx_user.join(tx_max_purchase[[\'customer_id\',\'recency\']],on=\'cus

In [0]:
# Scaling the variables
assembler_recency = vector_scaling(tx_user,"recency")
assembler_frequency = vector_scaling(tx_user,"frequency")
assembler_revenue = vector_scaling(tx_user,"revenue")

Exception in scaling function
recency does not exist. Available: customer_id
Exception in scaling function
frequency does not exist. Available: customer_id
Exception in scaling function
revenue does not exist. Available: customer_id

In [0]:
# Finding optimum recency clusters
finding_optimal_clusters(assembler_recency)

For 2 clusters, Silhouette with squared euclidean distance = 0.8499089343057348
For 3 clusters, Silhouette with squared euclidean distance = 0.8077035001703404
For 4 clusters, Silhouette with squared euclidean distance = 0.7474457848933599
For 5 clusters, Silhouette with squared euclidean distance = 0.7907700401255338
For 6 clusters, Silhouette with squared euclidean distance = 0.7704553376861911
For 7 clusters, Silhouette with squared euclidean distance = 0.7581770432514189
For 8 clusters, Silhouette with squared euclidean distance = 0.7614375901544942
For 9 clusters, Silhouette with squared euclidean distance = 0.7578304088182224
<Figure size 1440x576 with 1 Axes>

In [0]:
# Creating Clusters with default 6 clusters
recency_cluster = clustering(assembler_recency,6)
frequency_cluster = clustering(assembler_frequency,6)
revenue_cluster = clustering(assembler_revenue,6)

In [0]:
# Ordering recency cluster in Descending order
recency_ordered= recency_cluster.groupby('prediction').agg({'recency' : 'mean'}).toDF('prediction','recency').orderBy('recency')
recency_ordered = recency_ordered.selectExpr("row_number() over (order by recency desc) as recency_cluster", "*")
recency_ordered = recency_cluster.join(recency_ordered[['prediction','recency_cluster']],on=['prediction'],how='inner')
recency_ordered = recency_ordered.drop('prediction','features')

# Ordering frequency cluster in ascending order
frequency_ordered = frequency_cluster.groupby('prediction').agg({'frequency' : 'mean'}).toDF('prediction','frequency').orderBy('frequency')
frequency_ordered = frequency_ordered.selectExpr("row_number() over (order by frequency asc) as frequency_cluster", "*")
frequency_ordered = frequency_cluster.join(frequency_ordered[['prediction','frequency_cluster']],on=['prediction'],how='inner')
frequency_ordered = frequency_ordered.drop('prediction','features')

# Ordering revenue cluster in ascending order
revenue_ordered = revenue_cluster.groupby('prediction').agg({'revenue' : 'mean'}).toDF('prediction','revenue').orderBy('revenue')
revenue_ordered = revenue_ordered.selectExpr("row_number() over (order by revenue asc) as revenue_cluster", "*")
revenue_ordered = revenue_cluster.join(revenue_ordered[['prediction','revenue_cluster']],on=['prediction'],how='inner')
revenue_ordered = revenue_ordered.drop('prediction','features')

del(recency_cluster,frequency_cluster,revenue_cluster)
user_transaction_data = tx_user.join(recency_ordered[['customer_id','recency_cluster']],on=['customer_id'],how='left').join(frequency_ordered[['customer_id','frequency_cluster']],on=['customer_id'],how='left').join(revenue_ordered[['customer_id','revenue_cluster']],on=['customer_id'],how = 'left')

# finding overall score of the clusters.
user_transaction_data = user_transaction_data.withColumn('overall_score', expr("recency_cluster+frequency_cluster+revenue_cluster"))


In [0]:
# display(user_transaction_data.groupBy('overall_score').agg({'recency':'mean','frequency':'mean','revenue':'mean'}))

In [0]:
# So from above  results we can see 3 is the lowest and 18 is the highest score
# 3-5 = low value,6-13= Mid value, 13+ High value
user_transaction_data = user_transaction_data.withColumn('segment',when((col("overall_score")>5)&(col("overall_score")<=13),"mid-value").when((col("overall_score")>13)&(col("overall_score")<=18),"high-value").otherwise("low-value"))

In [0]:
# Creating dummy variables for segment and removing low-value to get rid of dummy variable trap
user_transaction_data = user_transaction_data.withColumn('seg-high-value',when(col("segment")=="high-value",1).otherwise(0))
user_transaction_data = user_transaction_data.withColumn('seg-mid-value',when(col("segment")=="mid-value",1).otherwise(0))

In [0]:
# display(user_transaction_data)

customer_id next_purchase_day_1st next_purchase_day_2nd next_purchase_day_3rd recency frequency revenue day_diff day_diff2 day_diff3 day_diff_mean day_diff_std recency_cluster frequency_cluster revenue_cluster overall_score segment seg-high-value seg-mid-value 20751931 94 99 261 82 19 2580.54 9 71 105 38.333333333333336 35.86248244417589 6 3 3 12 mid-value 0 1 32864555 431 511 525 284 10 532.09 330 420 434 91.88888888888889 103.14727873827361 4 2 1 7 mid-value 0 1 34710797 375 463 471 369 20 1160.51 20 22 142 40.10526315789474 33.886894774040606 3 3 2 8 mid-value 0 1 35270716 284 411 412 238 7 553.3199999999999 150 294 411 136.5 98.14020582819255 4 1 1 6 mid-value 0 1 37280532 475 507 514 411 6 513.24 5 103 187 41.8 45.339827966149144 3 1 1 5 low-value 0 0 38914426 428 443 475 203 8 336.97999999999996 2 502 517 104.57142857142857 182.77295308921495 5 1 1 7 mid-value 0 1 40798157 279 281 294 243 18 600.75 79 129 142 45.76470588235294 36.60179198441776 4 3 1 8 mid-value 0 1 105381265 367 368 526 313 8 903.11 62 210 211 70.85714285714286 63.370415209382394 3 1 1 5 low-value 0 0 108781378 80 87 132 70 27 1343.17 36 58 66 37.61538461538461 35.82633324589824 6 4 2 12 mid-value 0 1 122181737 37 189 215 26 12 1177.94 39 253 303 100.0909090909091 118.69242144758402 6 2 2 10 mid-value 0 1 141255312 140 141 217 111 13 1076.53 18 98 117 67.0 49.0361647617904 6 2 2 10 mid-value 0 1 141476897 173 179 182 125 15 675.5900000000001 55 68 281 71.85714285714286 70.84854609317317 6 2 1 9 mid-value 0 1 142082118 90 107 147 84 39 3210.78 15 24 39 27.5 25.431147108285213 6 4 3 13 mid-value 0 1 147634556 130 231 267 122 14 1383.23 80 137 157 76.0 54.477824234575785 6 2 2 10 mid-value 0 1 152535933 309 353 370 204 12 782.2400000000001 143 392 408 75.45454545454545 80.00920401599261 5 2 1 8 mid-value 0 1 171893700 517 574 677 512 10 526.8600000000001 4 146 194 61.111111111111114 60.364402681639376 2 2 1 5 low-value 0 0 178610360 459 489 560 278 11 442.70000000000005 120 364 395 85.9 88.03591438851659 4 2 1 7 mid-value 0 1 193051165 102 251 294 95 18 1537.2400000000002 14 56 123 59.35294117647059 48.27776555577964 6 3 2 11 mid-value 0 1 194612107 126 127 229 100 22 15390.0 25 109 110 34.666666666666664 51.44738412527242 6 3 6 15 high-value 1 0 198452835 170 216 249 62 11 107.22 4 57 281 99.4 108.84770196114489 6 2 1 9 mid-value 0 1 200311638 555 589 606 338 10 299.95000000000005 42 329 392 80.66666666666667 88.61010100434375 3 2 1 6 mid-value 0 1 207598620 465 497 526 360 12 953.5600000000001 41 236 264 46.18181818181818 51.578713015774596 3 2 1 6 mid-value 0 1 211925619 183 185 258 144 38 454.72000000000014 96 139 158 26.513513513513512 27.303460934082675 5 4 1 10 mid-value 0 1 215536017 182 319 352 154 18 2562.65 113 146 194 58.1764705882353 57.27917956609282 5 3 3 11 mid-value 0 1 218846997 284 293 303 68 15 602.25 39 172 278 76.64285714285714 89.40022985933034 6 2 1 9 mid-value 0 1 224674625 265 272 383 244 24 912.9600000000002 35 63 99 38.26086956521739 35.97501328738695 4 3 1 8 mid-value 0 1 226931899 105 120 129 70 38 2351.3700000000003 21 42 181 28.81081081081081 30.02576471210261 6 4 2 12 mid-value 0 1 235851854 93 106 133 76 15 809.15 9 18 56 70.5 86.02928839377007 6 2 1 9 mid-value 0 1 249604409 107 131 214 78 28 2251.4800000000005 43 60 78 30.814814814814813 35.89411462047036 6 4 2 12 mid-value 0 1 252874619 423 438 440 264 40 3499.499999999999 9 10 21 22.05128205128205 39.001989637472484 4 5 3 12 mid-value 0 1 261343110 497 549 584 316 7 1062.63 326 396 420 131.16666666666666 160.40002078137854 3 1 2 6 mid-value 0 1 263421120 91 95 116 77 35 4237.419999999999 36 95 105 30.205882352941178 31.657044227120462 6 4 3 13 mid-value 0 1 263565718 72 94 95 67 21 2918.7299999999996 28 85 92 54.0 63.848504907690426 6 3 3 12 mid-value 0 1 265191781 353 498 570 340 16 1770.6000000000001 164 325 375 53.13333333333333 54.34659493429539 3 3 2 8 mid-value 0 1 276173574 129 247 273 107 17 4215.54 33 103 165 64.3125 29.363738976272533 6 3 3 12 mid-value 0 1 2

In [0]:
X1 = user_transaction_data.drop('next_purchase_day_2nd','next_purchase_day_3rd','segment')
assembler_X1 = scaling(X1, X1.columns,["customer_id","next_purchase_day_1st"])
# train test splitting
train1, test1 = assembler_X1.randomSplit([0.7, 0.3], seed=42)

In [0]:
lr = LinearRegression(featuresCol = 'features', labelCol='next_purchase_day_1st', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train1)

In [0]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 54.674312
r2: 0.913843

In [0]:
lr_predictions = lr_model.transform(test1)
lr_predictions.select("prediction","next_purchase_day_1st").show(5)

+------------------+---------------------+
 prediction|next_purchase_day_1st|
+------------------+---------------------+
368.21706208794376| 431|
442.63330357008914| 375|
 292.7536700633762| 284|
138.32741962887468| 130|
358.39651176033726| 459|
+------------------+---------------------+
only showing top 5 rows

In [0]:
lr_evaluator = RegressionEvaluator(predictionCol="prediction",labelCol="next_purchase_day_1st",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

R Squared (R2) on test data = 0.914584

In [0]:
#del(X1)

In [0]:
user_transaction_data

Out[26]: DataFrame[customer_id: int, next_purchase_day_1st: int, next_purchase_day_2nd: int, next_purchase_day_3rd: int, recency: int, frequency: bigint, revenue: double, day_diff: int, day_diff2: int, day_diff3: int, day_diff_mean: double, day_diff_std: double, recency_cluster: int, frequency_cluster: int, revenue_cluster: int, overall_score: int, segment: string, seg-high-value: int, seg-mid-value: int]

In [0]:
#Selecting customers from user_transaction_data
customers = user_transaction_data.select('customer_id')

# Implementing Linear Regression on entire data for carrefour
#X1 = user_transaction_data.drop('2nd_next_purchase_day','3rd_next_purchase_day','4th_next_purchase_day','5th_next_purchase_day','segment')
#X2 = user_transaction_data.drop('1st_next_purchase_day','3rd_next_purchase_day','4th_next_purchase_day','5th_next_purchase_day','segment')
#X3 = user_transaction_data.drop('1st_next_purchase_day','2nd_next_purchase_day','4th_next_purchase_day','5th_next_purchase_day','segment')
#X4 = user_transaction_data.drop('1st_next_purchase_day','2nd_next_purchase_day','3rd_next_purchase_day','5th_next_purchase_day','segment')
#X5 = user_transaction_data.drop('1st_next_purchase_day','2nd_next_purchase_day','3rd_next_purchase_day','4th_next_purchase_day','segment')

# Implementing Linear Regression on entire data for ofc depot data
X1 = user_transaction_data.drop('next_purchase_day_2nd','next_purchase_day_3rd','segment')
X2 = user_transaction_data.drop('next_purchase_day_1st','next_purchase_day_3rd','segment')
X3 = user_transaction_data.drop('next_purchase_day_1st','next_purchase_day_2nd','segment')

#del(user_transaction_data)

In [0]:
# Scaling the data
assembler_X1 = scaling(X1, X1.columns,["customer_id","next_purchase_day_1st"])
assembler_X2 = scaling(X2, X2.columns,["customer_id","next_purchase_day_2nd"])
assembler_X3 = scaling(X3, X3.columns,["customer_id","next_purchase_day_3rd"])
#assembler_X4 = scaling(X4, X4.columns,["customer_id","4th_next_purchase_day"])
#assembler_X5 = scaling(X5, X5.columns,["customer_id","5th_next_purchase_day"])

#del(X1,X2,X3)
'''
# Implementing linear_regression
np1 = linear_regression(assembler_X1,"1st_next_purchase_day")
del(assembler_X1)
np2 = linear_regression(assembler_X2,"2nd_next_purchase_day")
del(assembler_X2)
np3 = linear_regression(assembler_X3,"3rd_next_purchase_day")
del(assembler_X3)
np4 = linear_regression(assembler_X4,"4th_next_purchase_day")
del(assembler_X4)
np5 = linear_regression(assembler_X5,"5th_next_purchase_day")
del(assembler_X5)
'''
linear_regression(assembler_X1,"next_purchase_day_1st").createOrReplaceTempView('np1')
linear_regression(assembler_X2,"next_purchase_day_2nd").createOrReplaceTempView('np2')
linear_regression(assembler_X3,"next_purchase_day_3rd").createOrReplaceTempView('np3')
#linear_regression(assembler_X4,"4th_next_purchase_day").createOrReplaceTempView('np4')
#linear_regression(assembler_X5,"5th_next_purchase_day").createOrReplaceTempView('np5')
#del(assembler_X1,assembler_X2,assembler_X3)

In [0]:
# Joining all predicted columns
customers.createOrReplaceTempView('customers')
final_predictions = spark.sql(""" 
select a.customer_id,next_purchase_day_1st,next_purchase_day_2nd,next_purchase_day_3rd
from customers as a left join np1 as b on a.customer_id = b.customer_id left join np2 as c on b.customer_id = c.customer_id
left join np3 as d on c.customer_id = d.customer_id  """)

spark.sql("drop view customers")
spark.sql("drop view np1")     
spark.sql("drop view np2")
spark.sql("drop view np3") 
#spark.sql("drop view np4")    
#spark.sql("drop view np5")    

#Creating purchases in to json format
#final_df = final_predictions.select("customer_id",to_json(struct("1st_next_purchase_day","2nd_next_purchase_day","3rd_next_purchase_day","4th_next_purchase_day","5th_next_purchase_day")).alias("days_next_purchase"))

Out[29]: DataFrame[]

In [0]:
display(final_predictions)

customer_id next_purchase_day_1st next_purchase_day_2nd next_purchase_day_3rd 19660317 163.29457182474897 193.11909815879397 232.310488740865 31757613 124.98860304442076 155.8491971416301 191.44962237220847 32258907 331.3514148576421 398.8130911278664 448.45353255462317 42564934 456.3653648658841 530.2411569585933 586.6773270788829 43414835 102.67263569319601 138.31692680495374 165.1760373892268 98957114 226.40662921393942 282.7731620525353 332.7680967181967 119190033 241.53013323102635 290.17398131726657 328.4676772149001 134261772 141.77762325824114 164.66630271218457 190.64367741048676 145944478 224.86000570862643 272.20406460572264 309.28703158872315 147823180 129.30715115561281 161.16402113412067 198.85701551994458 148110960 97.00102505803164 166.62021646335072 217.10495445146125 161642739 548.3065063448975 637.4759730069859 697.3447325136822 165584604 152.3234400133413 196.89901925639998 253.089600782076 167902143 409.3935142294349 490.11462232417034 542.5351756921405 175327169 109.39388641509902 158.16766682108516 195.91224082531414 181127038 512.839486068277 550.7289973672436 598.2983871799661 181594919 78.66956668836269 158.7815917205561 213.24010443678003 192451182 136.39324614735244 220.55699024909592 297.2339790031274 200799413 238.03085066456205 287.48217473424114 335.1363842305909 201403187 434.5378314080108 506.3470012035957 555.1561817352022 202051765 262.0813705664646 308.76599406347856 362.11592994627233 204035529 186.7866169868429 240.87137848018682 306.2867436425307 214013001 236.84065756674 285.7581982240921 333.9675964773808 215058911 174.46253284477405 258.8732386153319 308.60510682606827 216030374 103.72574789692194 140.26556634675737 163.474852195106 226853561 325.3966990830625 402.99212247470456 458.7670941478292 237221858 351.76632321649043 416.71093904643413 473.9795752855213 237719184 742.0972803877014 767.6649817182605 806.9313322669102 238906316 126.16359393768835 217.86105618730357 292.5766035341183 248824980 151.81631281806347 196.2613087544515 245.7986497426751 249560220 605.2180101417345 650.6624521526335 696.7811941444086 249616941 110.01773774861942 193.0370444292794 260.30329069253 252907322 124.84740753519998 184.81646445649093 227.59429360994977 258863033 584.2019785047344 643.632332775198 696.8045937025831 259811725 760.8401932945175 785.1335151187328 833.1948220352169 263403894 197.3178946326632 251.04226028689283 283.51330523156423 265837530 626.426567232291 659.9367017636561 702.0867460772414 274769998 186.24871046117852 262.4069614936387 314.6971998946524 276621944 361.09011517063516 407.8334259844003 457.33202006982685 278286703 123.5581717974286 176.1714588393666 228.22543441083522 280803850 369.83892158455365 419.6109719553312 470.0794333271513 282698698 147.6152542386282 180.4675742451678 220.475056810921 291697024 134.0295892427631 212.17901411132797 283.57577166597963 293256681 144.24671550614949 193.31530242813056 236.93107328053463 299332237 193.266361095651 256.1690139595024 295.7496781902669 305436464 190.97226388895737 233.85454926689596 262.4693290310392 311320126 209.51128841204434 288.2117357981647 349.3585287292974 316252906 99.07714951260215 141.6511585351166 167.97177317426116 318275802 195.42146384495248 257.614529867413 291.413391873849 320591893 140.02088555007737 169.89625414300087 211.66230790801615 321271810 176.86113101846723 195.41725803441278 233.29606727563376 323220374 138.3731766156426 189.58755590791722 241.19034994511935 332181462 255.13960196947895 312.28858211286837 353.8182648677184 332708192 267.61193660701707 301.7036011610013 349.58729417465065 332892848 244.12690568647832 302.8629505845287 358.72504092072666 335386679 125.70051818197624 171.86509821685704 212.41521973714313 341301791 370.7799311750351 482.2258851334924 539.7652652932504 342875354 115.44174175412564 171.21278753457113 211.48854480219754 346420701 210.05625361513472 271.9585938854315 321.8198320978607 347524168 195.57849469716098 255.91583949724827 296.33575247197126 352147262 115

In [0]:
final_predictions.write.mode("overwrite").option("path","/mnt/mountName/hive/warehouse/cpr_officedepot.db/customer_days_next_purchase").saveAsTable("cpr_officedepot.customer_days_next_purchase")

In [0]:
%sql
select * from cpr_officedepot.customer_days_next_purchase

customer_id next_purchase_day_1st next_purchase_day_2nd next_purchase_day_3rd 78677 144.939131182606 186.28283901508937 232.0875503951824 8229934 473.6657444055669 538.3475156656094 592.6317456461359 14248946 243.86788008034543 280.7440593786275 319.70856964897825 15297252 274.43957306584014 339.6545326015083 374.42231061542884 28280546 118.24862640575202 157.4054726577182 194.18032113158472 30778591 216.50739959811787 252.32396233333037 274.6771399222613 32923734 417.997964390563 519.1422939523757 580.9033643951253 33235857 250.09725322376818 295.98351216126895 342.97964612400585 37343898 343.33637003681434 403.01113124812395 455.26746557857285 41287981 102.76002151658685 143.07457961447145 163.7977088688849 42128763 236.266751691191 260.06273063436436 292.65118490501885 105100202 79.10882223989387 162.9223677820666 219.82354549594237 105572183 203.6589593533932 267.66659610733575 315.6902133192882 107652162 144.8673080599376 208.92468859298083 276.36825144224775 110304927 431.34941499431375 506.10032711946553 559.6896936470728 117778962 217.99737145584663 279.8123856165404 330.326575073693 127240619 156.91730366558522 190.5828218055658 232.9001393061302 147045361 579.0053083238797 639.9242364742203 690.0040573470563 166747402 139.6338302037181 180.13245892402736 228.44936305432853 175106131 117.75706208535502 143.86910721325023 172.62151659167432 183112160 111.49215431541757 155.1817090450679 179.27977768661182 186782500 587.844385691476 641.4727516486693 689.0249316806899 193680959 108.1646260604013 191.58759380412576 261.3560089592285 201412936 115.84616566954233 153.17231503473917 184.89720273703256 210028944 129.2322151722198 200.46440840948418 248.19424290620998 213261531 111.54089759052607 170.14692015495064 208.68339058678657 213973368 138.8402300079295 198.38306389576474 254.42386264048017 214010440 108.9891398866352 195.6018674870072 266.7036430538675 214405743 112.2136240932565 180.10924579021196 227.97567356029361 215033519 263.3869992264187 318.61138310347803 380.8310514184237 216614811 282.6702103084172 334.7627696093408 389.9628408301102 218747718 227.96229630688845 311.0118908478221 378.9119533246717 218777242 107.26377069083833 173.96173655823756 228.53406612905553 218812739 132.15654898837414 172.4641471455759 215.96638808682002 221587753 145.04491427750048 175.04166796182096 214.90979514924737 223199766 151.91133639735398 172.16567055286404 202.92755511867205 223287141 103.46940027330521 160.52774295146054 186.6258566288424 224663155 115.39605030645214 190.5268801508015 250.5254918173669 235307032 245.9873714048256 295.68089951041634 342.3017591466664 251930691 136.61275410290892 174.6527603079946 212.82688493227437 252421285 85.49864081916974 181.85749013081988 247.38117675640046 258126483 225.69793098973895 312.68319790968724 377.7706185717059 261348911 98.35246726528476 126.89954076649246 132.63683946902435 267896811 464.7549824426337 523.1822216040971 570.6785510229165 270007971 211.0770296331807 255.92472138968816 288.0020206603551 271683294 144.3863210226524 186.31139509380864 224.91734197456782 277851020 107.89580557871017 170.38634414677313 217.6969406951685 280293754 113.26190004742836 158.34518465343598 191.91120469990597 281084375 179.72962463946675 206.22873590453256 254.25663750564047 283268336 333.09121641071016 384.37458792065866 432.0362192826375 289833707 147.12450318659785 213.92090962091137 281.6870176221041 290544550 267.2296419233355 310.01213945396796 361.4008147975176 292231724 129.89388973200323 179.4799472678876 216.9599309447289 294817055 207.88918103589373 248.24008790191175 281.6738866903397 296508422 125.88732692047097 166.46648766400654 199.2472982851209 300199126 442.2821159295393 502.02244644197935 550.0920002035448 300217330 272.5317824539699 331.12168704819123 387.76068485047085 300225471 113.06695329531948 173.77943242462499 226.71731844747598 301857970 543.1119408728015 600.9229864860891 638.6459609499813 302966895 201.28951039264166 259.6263431946512 303.1885890682897 30